In [1]:
import time
import sys
sys.path.append('../')
import librairies.dagfeaturingfx 
from  librairies.dagfeaturingfx import *
from matplotlib import pyplot as plt
import seaborn as sns
from tqdm import tqdm
import joblib
from joblib import Parallel,delayed
import pyttsx3
from sklearn.preprocessing import MinMaxScaler,MaxAbsScaler,quantile_transform,PolynomialFeatures
from librairies.bt import *
from sklearn.metrics import accuracy_score, make_scorer, precision_score, recall_score, precision_recall_curve, confusion_matrix, classification_report
from sklearn import svm
from xgboost import XGBClassifier
from sklearn.metrics import f1_score
from sklearn.metrics import roc_curve
from numpy import sqrt
from numpy import argmax
import colorama as col
from collections import Counter

scaler = MinMaxScaler(feature_range=(0, 1))
engine = pyttsx3.init()
engine.say("Librairies loaded")
engine.runAndWait()

Importing Librairies...


###### RELOAD LIBRAIRIE #####
from importlib import reload
sys.path.append('../')
librairies.dagfeaturingfx= reload(librairies.dagfeaturingfx)
from  librairies.dagfeaturingfx import *

In [2]:
####### CHOIX DE LA TIME FRAME
_period = 'm15'

In [3]:
%%time
##### Récupération de la liste des tickers à partir du répertoire où sont rangées les bases
TICKER_LIST = list(sorted(set(get_ticker_list())))
TICKER_LIST = ['GBP/USD']


Récupération des tickers
CPU times: user 1.15 ms, sys: 1.81 ms, total: 2.96 ms
Wall time: 2.46 ms


In [4]:
#############################
##### F E A T U R I N G #####
#############################

In [5]:
%%time
df_all = joblib.load('JOBLIB/Built_bases/df_all')
daily_all = joblib.load('JOBLIB/Built_bases/daily_all')
weekly_all = joblib.load('JOBLIB/Built_bases/weekly_all')
df_all.shape,daily_all.shape,weekly_all.shape

CPU times: user 2.88 s, sys: 3.52 s, total: 6.4 s
Wall time: 6.4 s


((11069853, 56), (119160, 7), (23981, 19))

In [6]:
%%time
#### S T R A T E G Y

##### CONDITIONS LONG
_condition_1 = (df_all.slow_K5 < 20) & (df_all.slow_K5.shift(1) < df_all.slow_D5.shift(1)) & (df_all.slow_K5 > df_all.slow_D5)

##### CONDITIONS SHORT
_condition_1_bar = (df_all.slow_K5 > 80) & (df_all.slow_K5.shift(1) > df_all.slow_D5.shift(1)) & (df_all.slow_K5 < df_all.slow_D5)

##### 1 condition
df_all['Signal'] = np.where(_condition_1,1,np.where(_condition_1_bar,-1,0))



CPU times: user 565 ms, sys: 74.2 ms, total: 639 ms
Wall time: 377 ms


In [7]:
##### On backtest selon le ticker selectionné sur la période déterminée

x=TICKER_LIST[0]
print(x)
_year_bottom = '2010-01-01'
_year_top = '2017-12-31'
_nb_bougie_exit = 4200000000
_trigger_reengage = 0
_trigger_target = 1
_trigger_invers = 1
_trigger_sl = 1
_verbose = 0
_cash_ini = 200000
_rate = 1
backtest = df_all[df_all.Symbol == x.replace('/','')]
_target = 0.004
_exposure = 10
_size = 50000
_sl = 0.002
_open_hour = 8 # day only
_close_hour = 23 # day only
_window = 0 # day only

##### Backtest Over Night
TRACKER = bt(backtest,_year_bottom,_year_top,_nb_bougie_exit,_trigger_reengage,_trigger_target,_trigger_invers,_trigger_sl,_verbose,_cash_ini,\
        _rate,x,_target,_exposure,_size,_sl)

GBP/USD
Librairies imported

Début des opérations horodatée à 2021-02-05 16:05:31.415380

Chargement de la nouvelle base


 Le rate du ticker GBP/USD est à  1 
Bases chargées
TETEL process effectué
 ENTERING THE BACKTEST 
100%|██████████| 193628/193628 [00:07<00:00, 24491.89it/s]
 Pour le ticker  GBP/USD 
 
Les gains faramineux s'élèvent à : $  -31754.0 !. En  2920   transactions.
 
Nbre de winners : 1331 
 
Nbre de loosers : 1589 
Temps d'excution du BT 0:00:14.378045
 GBP/USD  results 
 Tested Period 2010-01-01  à 2017-12-31 
 Total Number of trades 2920 
Started Cash : 200000
P&L in currency:  -31754.0$ 
P&L in %:  -15.88% 
Average trade duration 32.1
# Winners  1331.0
# Loosers  1589.0
Cumulated gains 258100.0
Cumulated losses -289854.0
 PROFIT FACTOR :  0.89 
 Winners Ratio : 45.58 % 
Average Winners 193.91
% Average Winners 0.1
Average Loosers -182.41
% Average Loosers -0.09
Average pnl -10.87
% Average pnl -0.01
Number of opened trades 2920
Number of closed trades 2920
Max Expos

In [12]:
%%time  
df_all['TRACKER'] = np.where(df_all.index.isin(TRACKER),1,0)
df_all['Valid'] = np.where(((df_all.Signal!=0)&(df_all.TRACKER==1)),1,0)
df_all['Signal'] = np.where(((df_all.Valid==1)&(df_all.Signal==1)),1,np.where(((df_all.Valid==1)&(df_all.Signal==-1)),-1,0))
#df_all['Signal'] = np.where(df_all.index.isin(features[features.Signal==1].index),1,np.where(df_all.index.isin(features[features.Signal==-1].index),-1,0))
##### On backtest selon le ticker selectionné sur la période déterminée

x=TICKER_LIST[0]
print(x)
_year_bottom = '2010-01-01'
_year_top = '2017-12-31'
_nb_bougie_exit = 4200000000
_trigger_reengage = 0
_trigger_target = 1
_trigger_invers = 1
_trigger_sl = 1
_verbose = 0
_cash_ini = 200000
_rate = 1
backtest = df_all[df_all.Symbol == x.replace('/','')]
_target = 0.004
_exposure = 10
_size = 50000
_sl = 0.002
_open_hour = 8 # day only
_close_hour = 23 # day only
_window = 0 # day only

##### Backtest Over Night
TRACKER = bt(backtest,_year_bottom,_year_top,_nb_bougie_exit,_trigger_reengage,_trigger_target,_trigger_invers,_trigger_sl,_verbose,_cash_ini,\
        _rate,x,_target,_exposure,_size,_sl)

GBP/USD
Librairies imported

Début des opérations horodatée à 2021-02-05 16:08:37.310943

Chargement de la nouvelle base


 Le rate du ticker GBP/USD est à  1 
Bases chargées
TETEL process effectué
 ENTERING THE BACKTEST 
100%|██████████| 193628/193628 [00:05<00:00, 37058.71it/s]
 Pour le ticker  GBP/USD 
 
Les gains faramineux s'élèvent à : $  260930.5 !. En  757   transactions.
 
Nbre de winners : 757 
 
Nbre de loosers : 0 
Temps d'excution du BT 0:00:11.642027
 GBP/USD  results 
 Tested Period 2010-01-01  à 2017-12-31 
 Total Number of trades 757 
Started Cash : 200000
P&L  in currency:  260930.5$ 
P&L in %:  130.47% 
Average trade duration 59.89
# Winners  757.0
# Loosers  0.0
Cumulated gains 260930.5
Cumulated losses 0.0
 PROFIT FACTOR :  inf 
 Winners Ratio : 100.0 % 
Average Winners 344.69
% Average Winners 0.17
Pas de loosers
Average pnl 344.69
% Average pnl 0.3
Number of opened trades 757
Number of closed trades 757
Max Exposure 1 x  50000 =  50000 $
CPU times: user 6.87 s, s

In [13]:
%%time
df_all = joblib.load('JOBLIB/Built_bases/df_all')
df_all['Signal'] = np.where(_condition_1,1,np.where(_condition_1_bar,-1,0))
features = featuring(df_all)
features['TRACKER'] = np.where(features.index.isin(TRACKER),1,0)

CPU times: user 9.39 s, sys: 5.4 s, total: 14.8 s
Wall time: 10.7 s


In [14]:
%%time
# First, we must have an output. We'll call it 'Valid'. It wil be where Tracker & Signal are both to 1
features['Valid'] = np.where(((features.Signal!=0)&(features.TRACKER==1)),1,0) # Don't miss the point that even a Signal -1 must be considered as a good one by TRACKER

CPU times: user 103 ms, sys: 8.68 ms, total: 111 ms
Wall time: 55 ms


In [15]:
%%time
# Make our choice for the split, compliant to our backtest
_start = '2010-01-01' # start the train there
_mid = '2016-08-31' # stop the train and begin the test there
_stop = '2017-12-31' # stop the test there. After that, it is kept for oos

CPU times: user 3 µs, sys: 0 ns, total: 3 µs
Wall time: 6.2 µs


In [16]:
%%time
# Let's isolate the ticker on which we made the test
features = features[features.Symbol==x.replace('/','')]

# And drop the nan
features = features.dropna()
##### Signal is from strategy. This is potential good one. But we have to create the TRACKER column where the Signal where efficient

CPU times: user 3.1 s, sys: 2.34 s, total: 5.44 s
Wall time: 5.44 s


In [17]:
%%time
# Proceed an MaxAbsScaler on features
features_train,features_test,features_oos = scaling(features,x.replace('/',''),TRACKER,_start,_mid,_stop,scaler=MaxAbsScaler())


CPU times: user 323 ms, sys: 80.1 ms, total: 404 ms
Wall time: 402 ms


In [18]:
%%time
features_train,features_test,features_oos = quantile(features_train,features_test,features_oos,quantile_transform)

CPU times: user 504 ms, sys: 2.13 ms, total: 506 ms
Wall time: 505 ms


In [18]:
%%time
from sklearn.ensemble import GradientBoostingClassifier
_name = 'GradientBoostingClassifier'
_model = GradientBoostingClassifier(loss='deviance', learning_rate=0.1, n_estimators=100, subsample=1.0, criterion='friedman_mse', min_samples_split=2, min_samples_leaf=100, min_weight_fraction_leaf=0.0, max_depth=3, min_impurity_decrease=0.0, min_impurity_split=None, init=None, random_state=26, max_features=None, verbose=0, max_leaf_nodes=None, warm_start=False, validation_fraction=0.1, n_iter_no_change=None, tol=0.0001, ccp_alpha=0.0)

CPU times: user 20.7 ms, sys: 8.05 ms, total: 28.7 ms
Wall time: 27.5 ms


In [19]:
%%time
# from pycaret analysis
_name = 'GradientBoostingClassifier'
from sklearn.ensemble import GradientBoostingClassifier
_model = GradientBoostingClassifier(ccp_alpha=0.0, criterion='friedman_mse', init=None,
                           learning_rate=0.1, loss='deviance', max_depth=3,
                           max_features=None, max_leaf_nodes=None,
                           min_impurity_decrease=0.0, min_impurity_split=None,
                           min_samples_leaf=1, min_samples_split=2,
                           min_weight_fraction_leaf=0.0, n_estimators=100,
                           n_iter_no_change=None, presort='deprecated',
                           random_state=26, subsample=1.0, tol=0.0001,
                           validation_fraction=0.1, verbose=0,
                           warm_start=False)

CPU times: user 34 µs, sys: 0 ns, total: 34 µs
Wall time: 37.9 µs


In [20]:
%%time
from sklearn.tree import DecisionTreeClassifier
_name = 'DecisionTreeClassifier'
_model = DecisionTreeClassifier(criterion='gini', splitter='random', max_depth=None, min_samples_split=2, min_samples_leaf=1, min_weight_fraction_leaf=0.0, max_features='log2', random_state=26, max_leaf_nodes=None, min_impurity_decrease=0.0, min_impurity_split=None, class_weight=None, presort=True)

CPU times: user 31 µs, sys: 1e+03 ns, total: 32 µs
Wall time: 33.9 µs


In [19]:
%%time
from sklearn.neural_network import MLPClassifier
_name = 'MLPClassifier'
_model = MLPClassifier(hidden_layer_sizes=(200,8), activation='relu', solver='adam', alpha=0.000001, batch_size='auto', learning_rate='constant', learning_rate_init=0.001, power_t=0.5, max_iter=200, shuffle=True, random_state=26, tol=0.0000001, verbose=False, warm_start=True, momentum=0.9, nesterovs_momentum=True, early_stopping=False, validation_fraction=0.1, beta_1=0.8, beta_2=0.999, epsilon=1e-08)

CPU times: user 2.44 ms, sys: 2.65 ms, total: 5.09 ms
Wall time: 3.82 ms


In [20]:
%%time
_model.fit(features_train.drop(['Date','Symbol','TRACKER','Valid','Signal'],axis=1),features_train.Valid)
yhat = _model.predict(features_test.drop(['Date','Symbol','TRACKER','Valid','Signal'],axis=1))
accu = round(accuracy_score(features_test.Valid, yhat) * 100,2)
prec = round(precision_score(features_test.Valid, yhat,pos_label=1) * 100,2)
recall = round(recall_score(features_test.Valid, yhat) * 100,2)
f1 = round(f1_score(features_test.Valid, yhat) * 100,2)

print(col.Fore.BLUE,'Signaux pour',col.Fore.YELLOW,x,col.Style.RESET_ALL)
if accu > 69 and prec > 69 :
    print(col.Fore.GREEN)
elif accu < 51 or prec < 51 :
    print(col.Fore.RED)
else:
    print(col.Fore.YELLOW)

print('Signaux - Accuracy :' ,accu,'%')
print('Signaux - Precision :',prec,'%')
print('Signaux - Recall :', recall,'%')
print('Achat - F-measure: :' ,f1,'%',col.Style.RESET_ALL)
print('\n')
print(classification_report(features_test.Valid, yhat, target_names=['Neg', 'Pos']))
conf_matrix = pd.DataFrame(columns=['Bonnes_Estimations','Mauvaises_Estimations'])
tt = confusion_matrix(features_test.Valid, yhat, labels=[1,0])    #_model.classes_)
print('Vrais signaux trouvés    : ',col.Fore.GREEN,tt[0][0],col.Style.RESET_ALL)
print('Vrais signaux non trouvé :',col.Fore.RED,tt[0][1],col.Style.RESET_ALL)
print('Total des vrais signaux  :',col.Fore.BLUE,tt[0][0]+tt[0][1],col.Style.RESET_ALL)
print()
conf_matrix.loc['vrais-réels'] = tt[0]
conf_matrix.loc['faux-réels'] = tt[1]
savename = 'JOBLIB/'+_name+'/Save_'+x.replace('/','')+'_m15.sav'
joblib.dump(_model, savename)
print(col.Fore.BLUE,"Joblib Model ",savename," dumped to disk",col.Style.RESET_ALL)
conf_matrix

 Signaux pour  GBP/USD 

Signaux - Accuracy : 84.62 %
Signaux - Precision : 50.0 %
Signaux - Recall : 32.06 %
Achat - F-measure: : 39.07 % 


              precision    recall  f1-score   support

         Neg       0.88      0.94      0.91       721
         Pos       0.50      0.32      0.39       131

    accuracy                           0.85       852
   macro avg       0.69      0.63      0.65       852
weighted avg       0.83      0.85      0.83       852

Vrais signaux trouvés    :   42 
Vrais signaux non trouvé :  89 
Total des vrais signaux  :  131 

 Joblib Model  JOBLIB/MLPClassifier/Save_GBPUSD_m15.sav  dumped to disk 
CPU times: user 36.7 s, sys: 5.65 s, total: 42.3 s
Wall time: 5.34 s


,Bonnes_Estimations,Mauvaises_Estimations
vrais-réels,42,89
faux-réels,42,679


In [26]:
%%time
from sklearn.ensemble import RandomForestClassifier
_name = 'RandomForestClassifier'
_model = RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=-1, oob_score=False, random_state=26, verbose=0,
                       warm_start=False)

_model.fit(features_train.drop(['TRACKER','Symbol','Date','Valid','Signal'],axis=1),features_train.Valid)
yhat = _model.predict(features_test.drop(['TRACKER','Symbol','Date','Valid','Signal'],axis=1))
accu = round(accuracy_score(features_test.Valid, yhat) * 100,2)
prec = round(precision_score(features_test.Valid, yhat,pos_label=1) * 100,2)
recall = round(recall_score(features_test.Valid, yhat) * 100,2)
f1 = round(f1_score(features_test.Valid, yhat) * 100,2)

print(col.Fore.BLUE,'Achat pour',col.Fore.YELLOW,x,col.Style.RESET_ALL)
if accu > 69 and prec > 69 :
    print(col.Fore.GREEN)
elif accu < 51 or prec < 51 :
    print(col.Fore.RED)
else:
    print(col.Fore.YELLOW)

print('Achat - Accuracy :' ,accu,'%')
print('Achat - Precision :',prec,'%')
print('Achat - Recall :', recall,'%')
print('Achat - F-measure: :' ,f1,'%',col.Style.RESET_ALL)
print('\n')
print(classification_report(features_test.Valid, yhat, target_names=['Neg', 'Pos']))
conf_matrix = pd.DataFrame(columns=['Bonnes_Estimations','Mauvaises_Estimations'])
tt = confusion_matrix(features_test.Valid, yhat, labels=[1,0])    #_model.classes_)
print('Vrais signaux trouvés    : ',col.Fore.GREEN,tt[0][0],col.Style.RESET_ALL)
print('Vrais signaux non trouvé :',col.Fore.RED,tt[0][1],col.Style.RESET_ALL)
print('Total des vrais signaux  :',col.Fore.BLUE,tt[0][0]+tt[0][1],col.Style.RESET_ALL)
print()
conf_matrix.loc['vrais-réels'] = tt[0]
conf_matrix.loc['faux-réels'] = tt[1]
savename = 'JOBLIB/'+_name+'/Save_'+x.replace('/','')+'_m15.sav'
joblib.dump(_model, savename)
print(col.Fore.BLUE,"Joblib Model ",savename," dumped to disk",col.Style.RESET_ALL)
conf_matrix

 Achat pour  EUR/USD 

Achat - Accuracy : 87.93 %
Achat - Precision : 53.12 %
Achat - Recall : 14.41 %
Achat - F-measure: : 22.67 % 


              precision    recall  f1-score   support

         Neg       0.89      0.98      0.93       843
         Pos       0.53      0.14      0.23       118

    accuracy                           0.88       961
   macro avg       0.71      0.56      0.58       961
weighted avg       0.85      0.88      0.85       961

Vrais signaux trouvés    :   17 
Vrais signaux non trouvé :  101 
Total des vrais signaux  :  118 

 Joblib Model  JOBLIB/RandomForestClassifier/Save_EURUSD_m15.sav  dumped to disk 
CPU times: user 2.57 s, sys: 81.9 ms, total: 2.65 s
Wall time: 373 ms


,Bonnes_Estimations,Mauvaises_Estimations
vrais-réels,17,101
faux-réels,15,828


%%time
f_importance = pd.DataFrame(columns=features_train.drop(['Date','Symbol','TRACKER','Valid'],axis=1).columns.values.tolist()).T
f_importance['Importance'] = _model.feature_importances_
f_importance = f_importance
TOP_TEN = f_importance.sort_values('Importance',ascending=False).iloc[:12].index.to_list()
X_train = features_train[features_train.columns.intersection(TOP_TEN)]
y_train = features_train.Valid
X_test = features_test[features_test.columns.intersection(TOP_TEN)]
y_test = features_test.Valid
X_train.shape,X_test.shape,y_train.shape,y_test.shape
_model.fit(X_train,y_train)
yhat = _model.predict(X_test)
accu = round(accuracy_score(y_test, yhat) * 100,2)
prec = round(precision_score(y_test, yhat,pos_label=1) * 100,2)
recall = round(recall_score(y_test, yhat) * 100,2)
f1 = round(f1_score(y_test, yhat) * 100,2)

print(col.Fore.BLUE,'Achat pour',col.Fore.YELLOW,x,col.Style.RESET_ALL)
if accu > 69 and prec > 69 :
    print(col.Fore.GREEN)
elif accu < 51 or prec < 51 :
    print(col.Fore.RED)
else:
    print(col.Fore.YELLOW)

print('Achat - Accuracy :' ,accu,'%')
print('Achat - Precision :',prec,'%')
print('Achat - Recall :', recall,'%')
print('Achat - F-measure: :' ,f1,'%',col.Style.RESET_ALL)
print('\n')
print(classification_report(y_test, yhat, target_names=['Neg', 'Pos']))
conf_matrix = pd.DataFrame(columns=['Bonnes_Estimations','Mauvaises_Estimations'])
tt = confusion_matrix(y_test, yhat, labels=[1,0])    #_model.classes_)
print('Vrais signaux trouvés :',tt[0][0])
print('Vrais signaux non trouvé :',tt[0][1])
print('Total des vrais signaux :',tt[0][0]+tt[0][1])
print()
conf_matrix.loc['vrais-réels'] = tt[0]
conf_matrix.loc['faux-réels'] = tt[1]
conf_matrix

In [25]:
from sklearn.model_selection import GridSearchCV
clf = CatBoostClassifier()
params = {'iterations': [500],
          'depth': [5, 6],
          'loss_function': ['Logloss', 'CrossEntropy'],
          'l2_leaf_reg': np.logspace(-20, -19, 3),
          'leaf_estimation_iterations': [10,100,500],
#           'eval_metric': ['Accuracy'],
#           'use_best_model': ['True'],
          'logging_level':['Silent'],
          'random_seed': [42]
         }
scorer = make_scorer(precision_score,greater_is_better=True,  pos_label=0)
clf_grid = GridSearchCV(estimator=clf, param_grid=params, scoring=scorer, cv=5)

clf_grid.fit(features_train.drop(['Date','Symbol','TRACKER','Valid','Signal'],axis=1), features_train.Valid)
best_param = clf_grid.best_params_
best_param



{'depth': 5,
 'iterations': 500,
 'l2_leaf_reg': 1e-20,
 'leaf_estimation_iterations': 10,
 'logging_level': 'Silent',
 'loss_function': 'Logloss',
 'random_seed': 42}

In [27]:
%%time
from catboost import CatBoostClassifier
_name = 'CatBoostClassifier'

_model = CatBoostClassifier(
                             nan_mode = 'Min',
                             eval_metric = 'Logloss',
                                iterations = 1000,
                                sampling_frequency = 'PerTree',
                                leaf_estimation_method = 'Newton',
                                grow_policy = 'SymmetricTree',
                                penalties_coefficient = 1,
                                boosting_type = 'Plain',
                                model_shrink_mode = 'Constant',
                                feature_border_type = 'GreedyLogSum',
                                #bayesian_matrix_reg = 0.10000000149011612,
                                l2_leaf_reg = 3,
                                random_strength = 1,
                                rsm = 1,
                                boost_from_average = False,
                                model_size_reg = 0.5,
                                subsample = 0.800000011920929,
                                use_best_model = False,
                                class_names = [0,1],
                                random_seed = 42,
                                depth = 6,
                                posterior_sampling = False,
                                border_count = 254,
                                classes_count = 0,
                                auto_class_weights = None,
                                sparse_features_conflict_fraction = 0,
                                leaf_estimation_backtracking = 'AnyImprovement',
                                best_model_min_trees = 1,
                                model_shrink_rate = 0,
                                min_data_in_leaf = 1,
                                loss_function = 'Logloss',
                                learning_rate = 0.08382000029087067,
                                score_function = 'Cosine',
                                task_type = 'CPU',
                                leaf_estimation_iterations = 10,
                                bootstrap_type = 'MVS',
                                max_leaves = 64
)


_model.fit(features_train.drop(['Date','Symbol','TRACKER','Valid','Signal'],axis=1),y=features_train.Valid,verbose=0)


''',
    cat_features=None, 
    text_features=None,
    sample_weight=None,
    baseline=None,
    use_best_model=None,
    eval_set=None,
    verbose=False,
    logging_level=None,
    plot=False,
    column_description=None,
    verbose_eval=None,
    metric_period=None,
    silent=None,
    early_stopping_rounds=None,
    save_snapshot=None,
    snapshot_file=None,
    snapshot_interval=None,
    init_model=None)'''

yhat = _model.predict(features_test.drop(['Date','Symbol','TRACKER','Valid','Signal'],axis=1))
accu = round(accuracy_score(features_test.Valid, yhat) * 100,2)
prec = round(precision_score(features_test.Valid, yhat,pos_label=1) * 100,2)
recall = round(recall_score(features_test.Valid, yhat) * 100,2)
f1 = round(f1_score(features_test.Valid, yhat) * 100,2)

print(col.Fore.BLUE,'Achat pour',col.Fore.YELLOW,x,col.Style.RESET_ALL)
if accu > 69 and prec > 69 :
    print(col.Fore.GREEN)
elif accu < 51 or prec < 51 :
    print(col.Fore.RED)
else:
    print(col.Fore.YELLOW)

print('Achat - Accuracy :' ,accu,'%')
print('Achat - Precision :',prec,'%')
print('Achat - Recall :', recall,'%')
print('Achat - F-measure: :' ,f1,'%',col.Style.RESET_ALL)
print('\n')
print(classification_report(features_test.Valid, yhat, target_names=['Neg', 'Pos']))
conf_matrix = pd.DataFrame(columns=['Bonnes_Estimations','Mauvaises_Estimations'])
tt = confusion_matrix(features_test.Valid, yhat, labels=[1,0])    #_model.classes_)
print('Vrais signaux trouvés    : ',col.Fore.GREEN,tt[0][0],col.Style.RESET_ALL)
print('Vrais signaux non trouvé :',col.Fore.RED,tt[0][1],col.Style.RESET_ALL)
print('Total des vrais signaux  :',col.Fore.BLUE,tt[0][0]+tt[0][1],col.Style.RESET_ALL)
print()
conf_matrix.loc['vrais-réels'] = tt[0]
conf_matrix.loc['faux-réels'] = tt[1]
savename = 'JOBLIB/'+_name+'/Save_'+x.replace('/','')+'_m15.sav'
joblib.dump(_model, savename)
print(col.Fore.BLUE,"Joblib Model ",savename," dumped to disk",col.Style.RESET_ALL)
conf_matrix

 Achat pour  EUR/USD 

Achat - Accuracy : 87.1 %
Achat - Precision : 45.83 %
Achat - Recall : 27.97 %
Achat - F-measure: : 34.74 % 


              precision    recall  f1-score   support

         Neg       0.90      0.95      0.93       843
         Pos       0.46      0.28      0.35       118

    accuracy                           0.87       961
   macro avg       0.68      0.62      0.64       961
weighted avg       0.85      0.87      0.86       961

Vrais signaux trouvés    :   33 
Vrais signaux non trouvé :  85 
Total des vrais signaux  :  118 

 Joblib Model  JOBLIB/CatBoostClassifier/Save_EURUSD_m15.sav  dumped to disk 
CPU times: user 43.9 s, sys: 1.46 s, total: 45.3 s
Wall time: 4.31 s


,Bonnes_Estimations,Mauvaises_Estimations
vrais-réels,33,85
faux-réels,39,804


In [21]:
%%time
df_all_oos = df_all[df_all.index >= _stop]
df_all_oos = df_all_oos[df_all_oos.Symbol==x.replace('/','')].dropna()
df_all_oos['Valid'] = _model.predict(features_oos.drop(['Date','Symbol','Signal','TRACKER','Valid'],axis=1))

CPU times: user 1.51 s, sys: 1.2 s, total: 2.7 s
Wall time: 1.31 s


In [22]:
%%time
df_all_oos['Signal'] = np.where((df_all_oos.Signal==1)&(df_all_oos.Valid==1),1,np.where((df_all_oos.Signal==-1)&(df_all_oos.Valid==1),-1,0))

CPU times: user 14 ms, sys: 13.4 ms, total: 27.5 ms
Wall time: 3.36 ms


In [23]:
%%time  
df_all['TRACKER'] = np.where(df_all.index.isin(TRACKER),1,0)
df_all['Valid'] = np.where(((df_all.Signal!=0)&(df_all.TRACKER==1)),1,0)
df_all['Signal'] = np.where(((df_all.Valid==1)&(df_all.Signal==1)),1,np.where(((df_all.Valid==1)&(df_all.Signal==-1)),-1,0))
#df_all['Signal'] = np.where(df_all.index.isin(features[features.Signal==1].index),1,np.where(df_all.index.isin(features[features.Signal==-1].index),-1,0))
##### On backtest selon le ticker selectionné sur la période déterminée

x=TICKER_LIST[0]
print(x)
_year_bottom = '2018-01-01'
_year_top = '2020-12-31'
_nb_bougie_exit = 4200000000
_trigger_reengage = 0
_trigger_target = 1
_trigger_invers = 1
_trigger_sl = 1
_verbose = 0
_cash_ini = 200000
_rate = 1
_target = 0.004
_exposure = 10
_size = 50000
_sl = 0.002
_open_hour = 8 # day only
_close_hour = 23 # day only
_window = 0 # day only

##### Backtest Over Night
FINAL_TRACKER = bt(df_all_oos,_year_bottom,_year_top,_nb_bougie_exit,_trigger_reengage,_trigger_target,_trigger_invers,_trigger_sl,_verbose,_cash_ini,\
        _rate,x,_target,_exposure,_size,_sl)

GBP/USD
Librairies imported

Début des opérations horodatée à 2021-02-05 16:09:49.105386

Chargement de la nouvelle base


 Le rate du ticker GBP/USD est à  1 
Bases chargées
TETEL process effectué
 ENTERING THE BACKTEST 
100%|██████████| 72449/72449 [00:01<00:00, 44900.32it/s]
 Pour le ticker  GBP/USD 
 
Les gains faramineux s'élèvent à : $  43995.5 !. En  239   transactions.
 
Nbre de winners : 181 
 
Nbre de loosers : 58 
Temps d'excution du BT 0:00:07.908074
 GBP/USD  results 
 Tested Period 2018-01-01  à 2020-12-31 
 Total Number of trades 239 
Started Cash : 200000
P&L  in currency:  43995.5$ 
P&L in %:  22.0% 
Average trade duration 41.53
# Winners  181.0
# Loosers  58.0
Cumulated gains 53747.0
Cumulated losses -9751.5
 PROFIT FACTOR :  5.51 
 Winners Ratio : 75.73 % 
Average Winners 296.94
% Average Winners 0.15
Average Loosers -168.13
% Average Loosers -0.08
Average pnl 184.08
% Average pnl 0.11
Number of opened trades 239
Number of closed trades 239
Max Exposure 1 x  50000 = 

In [49]:
df_all_oos[df_all_oos.Signal==1].Signal.sum()

245